In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np           
import numpy
from numpy import array
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
stonks=['ADANIPORTS',	'ASIANPAINT',	'AXISBANK'	,'BAJAJ-AUTO',	'BAJFINANCE',	'BPCL'	,'BHARTIARTL',	'BRITANNIA'	,'CIPLA'	,'DIVISLAB',	'DRREDDY',	'GAIL',	'GRASIM',	'HCLTECH',	'HDFCBANK',	'HEROMOTOCO'	,'HINDALCO',	'HINDUNILVR',	'HDFC',	'ICICIBANK',	'ITC'	,'IOC'	,'INDUSINDBK'	,'INFY',	'JSWSTEEL'	,'KOTAKBANK',	'LT',	'M&M',	'NTPC',	'ONGC',	'POWERGRID'	,'RELIANCE',	'SBIN'	,'SUNPHARMA',	'TCS',	'TATAMOTORS',	'TATASTEEL',	'TECHM',	'TITAN',	'UPL',	'ULTRACEMCO',	'WIPRO']
len(stonks)                                       #list of nifty50 stocks after lagard removal

42

In [ ]:
x=pd.read_csv('/content/drive/My Drive/fyp/Lagard-less till 2021.csv',parse_dates=['Date'],index_col='Date')
x = x.loc["2016-01-01" :]                         #Since 2016-01-01, 5y(1238rows till 2020-12-31), + year 2021's rows
x.reset_index(drop=True, inplace=True)
x

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BPCL,BHARTIARTL,BRITANNIA,CIPLA,DIVISLAB,DRREDDY,GAIL,GRASIM,HCLTECH,HDFCBANK,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,NTPC,ONGC,POWERGRID,RELIANCE,SBIN,SUNPHARMA,TCS,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
0,267.55,878.75,449.90,2518.00,6095.85,896.05,340.50,2986.40,655.35,1162.50,3108.60,369.75,3767.75,845.85,1088.75,2686.05,84.90,856.55,1258.45,263.00,327.50,433.25,963.85,1105.25,1027.00,727.25,1289.20,1265.35,144.50,242.50,141.00,1015.35,227.80,815.55,2416.40,401.90,257.40,520.05,352.05,440.35,2824.00,556.45
1,257.95,880.80,438.40,2483.35,6052.45,891.45,326.80,2972.60,645.65,1149.70,3056.00,359.80,3657.25,845.95,1070.50,2637.80,80.75,859.00,1216.70,255.55,325.10,426.45,934.20,1078.90,1044.70,705.30,1255.95,1242.50,143.30,238.05,140.95,995.30,220.70,799.10,2369.60,377.05,256.90,516.40,351.40,433.85,2749.15,557.70
2,255.70,900.65,436.45,2502.35,5997.85,902.65,323.45,2966.70,640.85,1147.50,3070.65,372.25,3659.10,842.80,1062.40,2616.85,82.60,847.95,1209.40,256.70,324.85,442.65,935.25,1074.05,1083.10,707.75,1256.90,1249.20,142.25,241.85,139.15,1005.15,217.75,800.50,2348.95,374.45,274.30,521.90,347.70,435.30,2733.05,556.90
3,251.80,886.40,430.70,2485.50,6107.25,912.30,322.20,3001.70,652.10,1137.20,3046.25,372.70,3652.40,841.40,1067.10,2578.20,80.55,843.05,1209.30,250.10,315.10,450.85,943.95,1069.35,1058.20,703.60,1236.85,1226.15,141.45,237.55,139.50,1032.20,216.85,791.95,2381.60,365.90,268.75,525.10,347.85,443.35,2735.30,555.10
4,245.00,872.35,409.25,2419.25,6106.30,884.25,322.35,2946.55,637.45,1126.95,2991.80,370.60,3557.20,825.05,1056.20,2520.10,76.65,820.25,1179.45,246.75,309.60,441.55,925.50,1050.80,1026.35,691.65,1206.40,1197.45,137.85,226.60,138.10,1013.25,209.55,783.80,2371.25,343.55,249.90,510.80,344.05,417.40,2667.70,549.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,499.45,2793.85,664.45,3492.65,5119.00,392.00,514.00,3551.10,827.25,3842.10,5286.90,129.30,961.30,991.35,1426.70,3067.20,250.30,2450.55,2651.85,537.25,211.45,93.00,921.65,1293.80,395.25,1959.75,1306.30,740.10,97.60,94.95,188.00,1966.10,281.75,603.45,3093.00,193.20,680.55,1003.85,1570.95,471.25,5341.20,406.30
1241,496.80,2805.35,654.25,3462.70,5030.30,387.30,525.30,3539.70,824.80,3879.85,5288.30,134.00,984.60,978.20,1420.55,3083.55,259.05,2417.30,2638.85,546.70,205.40,94.10,922.35,1282.10,401.70,1970.40,1314.00,736.10,97.85,96.95,196.15,1914.25,285.05,605.30,3051.50,195.40,683.80,997.15,1572.60,472.40,5448.35,406.40
1242,513.85,2792.25,671.10,3437.95,5081.00,392.20,545.25,3552.80,826.55,3803.05,5270.90,133.45,993.85,962.55,1416.25,3055.25,272.90,2368.85,2661.35,541.10,202.80,94.50,952.05,1262.15,405.40,1952.40,1338.95,744.40,97.00,97.90,197.05,1911.15,287.70,601.90,3032.80,196.75,722.80,994.75,1542.35,482.50,5397.95,406.75
1243,517.00,2844.70,672.70,3529.15,5082.00,400.15,540.25,3575.25,838.70,3859.15,5338.25,132.75,1004.30,994.65,1431.65,3161.10,268.20,2391.20,2657.50,542.05,201.50,96.15,939.80,1312.10,402.85,1970.70,1373.40,770.50,100.15,100.65,203.55,1933.70,286.00,620.80,3120.90,198.15,713.15,1051.10,1548.60,503.65,5591.75,430.20


In [ ]:
alldata=x   #the historical dataset

In [ ]:
timesteps=30                                     #lstm hyperparameters "Subject to be tuned"
epoch=50
batchSize=64
neurons=50
after2020=len(alldata)-1238                      #number of days after 31-12-2020

In [ ]:
def create_dataset(dataset, time_step=1):         # convert an array of values into a dataset matrix which will be used to train the lstm model.
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step):
		a = dataset[i:(i+time_step), 0]               #i=0, 0,1,2,3-----99   100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
def forcast(df1, timesteps, epoch, batchSize, neurons, after2020):             
  scaler=MinMaxScaler(feature_range=(0,1))   
  df1=scaler.fit_transform(np.array(df1).reshape(-1,1))           #minmax scalar transformation of data

  training_size=int(len(df1)-after2020)                                
  train_data=df1[0:training_size,:]                               #getting the training data
  inpdata=df1[training_size-timesteps:len(df1),:1]                #getting the data from 01-01-2021 onwards
  val_data=train_data[int(training_size*0.75):len(train_data),:1] #validation data (25% of training data)

  time_step = timesteps                                           #reshape into X=t,t+1,t+2,t+3,........t+"timestep-1" and Y=t+"timestep"
  X_train, y_train = create_dataset(train_data, time_step)
  x_inp, y_inp = create_dataset(inpdata, timesteps)
  x_val, y_val = create_dataset(val_data,timesteps)

  X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1) 
  x_inp = x_inp.reshape(x_inp.shape[0],x_inp.shape[1] , 1)        #reshape input to be [samples, time steps, features] which is required for LSTM
  x_val = x_val.reshape(x_val.shape[0],x_val.shape[1] , 1)

  # initialising stacked lstm
  model=Sequential()
  model.add(LSTM(neurons,return_sequences=True,input_shape=(time_step,1)))
  model.add(LSTM(neurons,return_sequences=True))
  model.add(LSTM(neurons))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',optimizer='adam')

  model.fit(X_train,y_train,validation_data=(x_val,y_val),epochs=epoch,batch_size=batchSize,verbose=1)     # training of the model
  
  out_predict=model.predict(x_inp)                      #dynamic prediction of the stock's closing price from 01-01-2021 onwards

  v=math.sqrt(mean_squared_error(y_inp,out_predict))    # RMSE value calculated here

  lst=[]
  for i in out_predict:
    lst.append(i)

  df3=train_data.tolist()
  df3.extend(lst)                                       #appending the data, forcasted from 01-01-2021 onwards, to the data till 31-12-2020
  df3=scaler.inverse_transform(df3).tolist()

  return pd.DataFrame(df3),v
    

In [ ]:
rmse=[]
fdata=pd.DataFrame()
for i in alldata:                                   # this for loop will be iterated for 42 times i.e. for each column of the historical dataset
  temp=alldata[i]
  ftemp,trmse=forcast(temp, timesteps, epoch, batchSize, neurons, after2020)    #hyperparameters are provided as input here
  fdata = pd.concat([fdata,ftemp],axis = 1)
  rmse.append(trmse)
fdata.columns=stonks

19/19 [==============================] - 1s 60ms/step - loss: 0.0026 - val_loss: 0.0042
Epoch 23/50
19/19 [==============================] - 1s 62ms/step - loss: 0.0027 - val_loss: 0.0041
Epoch 24/50
19/19 [==============================] - 1s 60ms/step - loss: 0.0024 - val_loss: 0.0039
Epoch 25/50
19/19 [==============================] - 1s 63ms/step - loss: 0.0023 - val_loss: 0.0037
Epoch 26/50
19/19 [==============================] - 1s 62ms/step - loss: 0.0023 - val_loss: 0.0039
Epoch 27/50
19/19 [==============================] - 1s 62ms/step - loss: 0.0021 - val_loss: 0.0036
Epoch 28/50
19/19 [==============================] - 1s 63ms/step - loss: 0.0022 - val_loss: 0.0036
Epoch 29/50
19/19 [==============================] - 1s 62ms/step - loss: 0.0020 - val_loss: 0.0036
Epoch 30/50
19/19 [==============================] - 1s 64ms/step - loss: 0.0018 - val_loss: 0.0033
Epoch 31/50
19/19 [==============================] - 1s 63ms/step - loss: 0.0021 - val_loss: 0.0031
Epoch 32/50


In [ ]:
fdata # forcasted dataset

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BPCL,BHARTIARTL,BRITANNIA,CIPLA,DIVISLAB,DRREDDY,GAIL,GRASIM,HCLTECH,HDFCBANK,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,NTPC,ONGC,POWERGRID,RELIANCE,SBIN,SUNPHARMA,TCS,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
0,267.550000,878.750000,449.900000,2518.000000,6095.850000,896.050000,340.500000,2986.400000,655.350000,1162.500000,3108.600000,369.750000,3767.750000,845.850000,1088.750000,2686.050000,84.900000,856.550000,1258.450000,263.000000,327.500000,433.250000,963.850000,1105.250000,1027.000000,727.250000,1289.200000,1265.350000,144.500000,242.500000,141.000000,1015.350000,227.800000,815.550000,2416.400000,401.900000,257.400000,520.050000,352.050000,440.350000,2824.000000,556.450000
1,257.950000,880.800000,438.400000,2483.350000,6052.450000,891.450000,326.800000,2972.600000,645.650000,1149.700000,3056.000000,359.800000,3657.250000,845.950000,1070.500000,2637.800000,80.750000,859.000000,1216.700000,255.550000,325.100000,426.450000,934.200000,1078.900000,1044.700000,705.300000,1255.950000,1242.500000,143.300000,238.050000,140.950000,995.300000,220.700000,799.100000,2369.600000,377.050000,256.900000,516.400000,351.400000,433.850000,2749.150000,557.700000
2,255.700000,900.650000,436.450000,2502.350000,5997.850000,902.650000,323.450000,2966.700000,640.850000,1147.500000,3070.650000,372.250000,3659.100000,842.800000,1062.400000,2616.850000,82.600000,847.950000,1209.400000,256.700000,324.850000,442.650000,935.250000,1074.050000,1083.100000,707.750000,1256.900000,1249.200000,142.250000,241.850000,139.150000,1005.150000,217.750000,800.500000,2348.950000,374.450000,274.300000,521.900000,347.700000,435.300000,2733.050000,556.900000
3,251.800000,886.400000,430.700000,2485.500000,6107.250000,912.300000,322.200000,3001.700000,652.100000,1137.200000,3046.250000,372.700000,3652.400000,841.400000,1067.100000,2578.200000,80.550000,843.050000,1209.300000,250.100000,315.100000,450.850000,943.950000,1069.350000,1058.200000,703.600000,1236.850000,1226.150000,141.450000,237.550000,139.500000,1032.200000,216.850000,791.950000,2381.600000,365.900000,268.750000,525.100000,347.850000,443.350000,2735.300000,555.100000
4,245.000000,872.350000,409.250000,2419.250000,6106.300000,884.250000,322.350000,2946.550000,637.450000,1126.950000,2991.800000,370.600000,3557.200000,825.050000,1056.200000,2520.100000,76.650000,820.250000,1179.450000,246.750000,309.600000,441.550000,925.500000,1050.800000,1026.350000,691.650000,1206.400000,1197.450000,137.850000,226.600000,138.100000,1013.250000,209.550000,783.800000,2371.250000,343.550000,249.900000,510.800000,344.050000,417.400000,2667.700000,549.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,491.274694,2928.861454,633.258949,3453.281130,5261.577293,376.306029,517.094406,3548.386239,839.259267,3741.909291,5184.407090,122.455771,918.113701,956.527491,1421.980039,3073.712970,239.984211,2328.942953,2562.821015,542.579025,208.451654,90.393951,875.289496,1233.586867,387.521254,1969.456457,1324.225139,732.642668,98.829868,92.660906,189.619047,2008.620153,283.332065,588.484074,2964.145698,182.208902,636.997120,977.273507,1564.678269,464.228954,5412.951857,405.884540
1241,494.365585,2950.587272,642.454811,3460.531913,5214.271489,378.794869,517.024177,3541.970118,840.530423,3763.292511,5211.008466,123.581692,926.237696,970.705303,1424.306567,3062.645048,243.680047,2343.034020,2591.699922,542.996305,209.260195,90.759617,884.620764,1249.513140,391.210068,1954.203619,1330.442264,738.977534,98.655735,92.840294,189.297551,1997.499336,284.503609,591.692702,3076.016313,185.239331,649.198232,986.321778,1568.277384,467.710173,5423.690146,411.751053
1242,494.077483,2968.564990,652.714491,3450.429113,5131.973948,380.679815,520.956161,3534.772118,840.577841,3780.892833,5230.635524,125.142222

In [ ]:
alldata # actual dataset

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BPCL,BHARTIARTL,BRITANNIA,CIPLA,DIVISLAB,DRREDDY,GAIL,GRASIM,HCLTECH,HDFCBANK,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,NTPC,ONGC,POWERGRID,RELIANCE,SBIN,SUNPHARMA,TCS,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
0,267.55,878.75,449.90,2518.00,6095.85,896.05,340.50,2986.40,655.35,1162.50,3108.60,369.75,3767.75,845.85,1088.75,2686.05,84.90,856.55,1258.45,263.00,327.50,433.25,963.85,1105.25,1027.00,727.25,1289.20,1265.35,144.50,242.50,141.00,1015.35,227.80,815.55,2416.40,401.90,257.40,520.05,352.05,440.35,2824.00,556.45
1,257.95,880.80,438.40,2483.35,6052.45,891.45,326.80,2972.60,645.65,1149.70,3056.00,359.80,3657.25,845.95,1070.50,2637.80,80.75,859.00,1216.70,255.55,325.10,426.45,934.20,1078.90,1044.70,705.30,1255.95,1242.50,143.30,238.05,140.95,995.30,220.70,799.10,2369.60,377.05,256.90,516.40,351.40,433.85,2749.15,557.70
2,255.70,900.65,436.45,2502.35,5997.85,902.65,323.45,2966.70,640.85,1147.50,3070.65,372.25,3659.10,842.80,1062.40,2616.85,82.60,847.95,1209.40,256.70,324.85,442.65,935.25,1074.05,1083.10,707.75,1256.90,1249.20,142.25,241.85,139.15,1005.15,217.75,800.50,2348.95,374.45,274.30,521.90,347.70,435.30,2733.05,556.90
3,251.80,886.40,430.70,2485.50,6107.25,912.30,322.20,3001.70,652.10,1137.20,3046.25,372.70,3652.40,841.40,1067.10,2578.20,80.55,843.05,1209.30,250.10,315.10,450.85,943.95,1069.35,1058.20,703.60,1236.85,1226.15,141.45,237.55,139.50,1032.20,216.85,791.95,2381.60,365.90,268.75,525.10,347.85,443.35,2735.30,555.10
4,245.00,872.35,409.25,2419.25,6106.30,884.25,322.35,2946.55,637.45,1126.95,2991.80,370.60,3557.20,825.05,1056.20,2520.10,76.65,820.25,1179.45,246.75,309.60,441.55,925.50,1050.80,1026.35,691.65,1206.40,1197.45,137.85,226.60,138.10,1013.25,209.55,783.80,2371.25,343.55,249.90,510.80,344.05,417.40,2667.70,549.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,499.45,2793.85,664.45,3492.65,5119.00,392.00,514.00,3551.10,827.25,3842.10,5286.90,129.30,961.30,991.35,1426.70,3067.20,250.30,2450.55,2651.85,537.25,211.45,93.00,921.65,1293.80,395.25,1959.75,1306.30,740.10,97.60,94.95,188.00,1966.10,281.75,603.45,3093.00,193.20,680.55,1003.85,1570.95,471.25,5341.20,406.30
1241,496.80,2805.35,654.25,3462.70,5030.30,387.30,525.30,3539.70,824.80,3879.85,5288.30,134.00,984.60,978.20,1420.55,3083.55,259.05,2417.30,2638.85,546.70,205.40,94.10,922.35,1282.10,401.70,1970.40,1314.00,736.10,97.85,96.95,196.15,1914.25,285.05,605.30,3051.50,195.40,683.80,997.15,1572.60,472.40,5448.35,406.40
1242,513.85,2792.25,671.10,3437.95,5081.00,392.20,545.25,3552.80,826.55,3803.05,5270.90,133.45,993.85,962.55,1416.25,3055.25,272.90,2368.85,2661.35,541.10,202.80,94.50,952.05,1262.15,405.40,1952.40,1338.95,744.40,97.00,97.90,197.05,1911.15,287.70,601.90,3032.80,196.75,722.80,994.75,1542.35,482.50,5397.95,406.75
1243,517.00,2844.70,672.70,3529.15,5082.00,400.15,540.25,3575.25,838.70,3859.15,5338.25,132.75,1004.30,994.65,1431.65,3161.10,268.20,2391.20,2657.50,542.05,201.50,96.15,939.80,1312.10,402.85,1970.70,1373.40,770.50,100.15,100.65,203.55,1933.70,286.00,620.80,3120.90,198.15,713.15,1051.10,1548.60,503.65,5591.75,430.20


In [ ]:
rmse                                  #between original data(after 31-12-2020) and dynamically predicted data (after 31-12-2020)

[0.042989807410568785,
 0.0702256782142951,
 0.030288418632676003,
 0.05233953442362253,
 0.0089872430052715,
 0.017278831331289653,
 0.033818823265396425,
 0.007362592398801193,
 0.024016429131970593,
 0.031593480520855616,
 0.029711734138390226,
 0.015646008431992924,
 0.010330101415843793,
 0.045437015023994616,
 0.008186619164917922,
 0.02888507233075661,
 0.06563340355807318,
 0.046986049277614635,
 0.032879400107791265,
 0.017760179183962214,
 0.024913043723666182,
 0.005754544171136107,
 0.02308043386781796,
 0.07512688942347971,
 0.006183528222363233,
 0.013152633166523172,
 0.01449810948781094,
 0.01420521024036168,
 0.009996273825086678,
 0.019788315876967184,
 0.07281704546595948,
 0.030866729423332014,
 0.017163819575954368,
 0.031633249891608456,
 0.0456528294365567,
 0.025888083266432507,
 0.07644431980065283,
 0.05451347532436734,
 0.011288648612364054,
 0.020185110753489166,
 0.024887461490161904,
 0.0231213136449041]

In [ ]:
fdata.to_csv('/content/drive/My Drive/fyp/fdata.csv')   #forcasted dataset saved in .csv format